In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib as mat
import matplotlib.pyplot as plt
import seaborn as sns
import xlrd
import platform
import json
import googlemaps
import folium
from matplotlib import font_manager , rc
f_path = "/Library/Fonts/Arial Unicode.ttf"
font_name = font_manager.FontProperties(fname=f_path).get_name()
rc('font', family=font_name)

In [2]:
crime = pd.read_excel('../data/서울시 범죄건수.xlsx')
crime = crime[['자치구별(2)', '2017', '2018', '2019']]
crime.rename(columns = {'자치구별(2)' : '시군구'}, inplace = True) # 이름 변경

/Users/jeongjinseo/Desktop/myweedsgroweveryday/playdata/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
pop = pd.read_excel('../data/주민등록인구_20230711160850.xlsx')
pop = pop[['동별(2)', '2017', '2018', '2019']]

/Users/jeongjinseo/Desktop/myweedsgroweveryday/playdata/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
pop = pop.drop(range(0, 2)).reset_index(drop = True)
pop['시도'] = '서울특별시'

In [5]:
pop

,동별(2),2017,2018,2019,시도
0,종로구,164257,163026,161869,서울특별시
1,중구,134593,135633,136488,서울특별시
2,용산구,244444,245090,245185,서울특별시
3,성동구,312711,316463,308979,서울특별시
4,광진구,372298,371063,366972,서울특별시
5,동대문구,366011,364338,363023,서울특별시
6,중랑구,412780,408147,402024,서울특별시
7,성북구,455407,447687,454744,서울특별시
8,강북구,328002,322915,317695,서울특별시
9,도봉구,346234,341649,335631,서울특별시


In [6]:
pop = pop.rename(columns = {'동별(2)' : '시군구', '2017' : '2017 인구합계','2018' : '2018 인구합계','2019' : '2019 인구합계'})
pop = pop[['시도', '시군구', '2017 인구합계', '2018 인구합계', '2019 인구합계']]
pop

,시도,시군구,2017 인구합계,2018 인구합계,2019 인구합계
0,서울특별시,종로구,164257,163026,161869
1,서울특별시,중구,134593,135633,136488
2,서울특별시,용산구,244444,245090,245185
3,서울특별시,성동구,312711,316463,308979
4,서울특별시,광진구,372298,371063,366972
5,서울특별시,동대문구,366011,364338,363023
6,서울특별시,중랑구,412780,408147,402024
7,서울특별시,성북구,455407,447687,454744
8,서울특별시,강북구,328002,322915,317695
9,서울특별시,도봉구,346234,341649,335631


In [7]:
crime = crime.drop(range(0, 4)).reset_index(drop = True)
crime['시도'] = '서울특별시'
crime

,시군구,2017,2018,2019,시도
0,종로구,4057,3690,3846,서울특별시
1,중구,4184,4030,4327,서울특별시
2,용산구,4060,3411,3313,서울특별시
3,성동구,2767,2457,2512,서울특별시
4,광진구,4646,3915,4011,서울특별시
5,동대문구,3975,3680,3692,서울특별시
6,중랑구,4571,4288,4268,서울특별시
7,성북구,3434,3042,2877,서울특별시
8,강북구,3393,3437,3838,서울특별시
9,도봉구,1999,2249,2110,서울특별시


In [8]:
crime = crime[['시도', '시군구', '2017', '2018', '2019']]

In [9]:
crime = crime.rename(columns = {'2017' : '2017 범죄발생합계', '2018' : '2018 범죄발생합계', '2019' : '2019 범죄발생합계'})

In [10]:
crime

,시도,시군구,2017 범죄발생합계,2018 범죄발생합계,2019 범죄발생합계
0,서울특별시,종로구,4057,3690,3846
1,서울특별시,중구,4184,4030,4327
2,서울특별시,용산구,4060,3411,3313
3,서울특별시,성동구,2767,2457,2512
4,서울특별시,광진구,4646,3915,4011
5,서울특별시,동대문구,3975,3680,3692
6,서울특별시,중랑구,4571,4288,4268
7,서울특별시,성북구,3434,3042,2877
8,서울특별시,강북구,3393,3437,3838
9,서울특별시,도봉구,1999,2249,2110


In [11]:
pop_crime = crime.merge(pop, on = '시군구', how = 'outer')
pop_crime = pop_crime.drop(['시도_y'], axis = 1)

In [12]:
pop_crime = pop_crime.rename(columns = {'시도_x' : '시도'})

In [13]:
pop_crime = pop_crime.sort_values(by = '시군구').reset_index(drop = True)

In [14]:
pop_crime.to_excel('../data/서울시_구별_범죄발생_인구수.xlsx')